# Conservative remapping

In [2]:
import xgcm
import xarray as xr
import numpy as np
import xbasin

We open the example data and create 2 grids: 1 for the dataset we have and 1 for the remapped one.
Here '_fr' means *from* and '_to' *to* (i.e. remapped data).

In [11]:
ds = xr.open_dataset('data/nemo_output_ex.nc')
metrics_fr = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'],
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'],
    ('Z',): ['e3t', 'e3u', 'e3v', 'e3w']
}
metrics_to = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'],
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'],
    ('Z',): ['e3t_1d', 'e3w_1d']
}
grid_fr = xgcm.Grid(ds, periodic=False, metrics=metrics_fr)
grid_to = xgcm.Grid(ds, periodic=False, metrics=metrics_to)

print(ds)

<xarray.Dataset>
Dimensions:        (axis_nbounds: 2, t: 1, x_c: 20, x_f: 20, y_c: 40, y_f: 40, z_c: 36, z_f: 36)
Coordinates:
  * z_f            (z_f) float64 -0.5 0.5 1.5 2.5 3.5 ... 31.5 32.5 33.5 34.5
  * t              (t) object 1050-07-01 00:00:00
  * x_c            (x_c) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19
  * y_c            (y_c) int64 0 1 2 3 4 5 6 7 8 ... 31 32 33 34 35 36 37 38 39
  * z_c            (z_c) int64 0 1 2 3 4 5 6 7 8 ... 27 28 29 30 31 32 33 34 35
  * x_f            (x_f) float64 0.5 1.5 2.5 3.5 4.5 ... 16.5 17.5 18.5 19.5
  * y_f            (y_f) float64 0.5 1.5 2.5 3.5 4.5 ... 36.5 37.5 38.5 39.5
Dimensions without coordinates: axis_nbounds
Data variables:
    depthw_bounds  (z_f, axis_nbounds) float32 ...
    t_bounds       (t, axis_nbounds) object ...
    e3w            (t, z_f, y_c, x_c) float64 ...
    woce           (t, z_f, y_c, x_c) float64 ...
    deptht_bounds  (z_c, axis_nbounds) float32 ...
    e3t            (t, z_c, y_c, x_c) f

We remap

In [13]:
theta_to = xbasin.remap_vertical(ds.thetao, grid_fr, grid_to, axis='Z', z_fr=ds.gdepw_0, z_to=ds.gdepw_1d)
print(theta_to.head())

<xarray.DataArray 'thetao' (t: 1, z_c: 5, y_c: 5, x_c: 5)>
array([[[[ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        , 24.31871114, 24.35898565, 24.41627737,
          24.41330382],
         [ 0.        , 24.27254829, 24.33234014, 24.39657273,
          24.45444904],
         [ 0.        , 24.26260741, 24.36464639, 24.25571325,
          24.43841119],
         [ 0.        , 24.22003897, 24.34994081, 24.10271889,
          24.42661558]],

        [[ 0.        ,  0.        ,  0.        ,  0.        ,
           0.        ],
         [ 0.        , 24.29409441, 24.33487837, 24.39324951,
          24.39125943],
         [ 0.        , 24.25011105, 24.30913507, 24.37540212,
          24.43508995],
         [ 0.        , 24.25146374, 24.34960859, 24.23679286,
          24.42124178],
         [ 0.        , 24.21078069, 24.34134889, 24.08688472,
          24.41539059]],

        [[ 0.        ,  0.        ,  0.        ,  0.        ,
           0.  

The total heat content is conserved:

In [10]:
hc_fr = grid_fr.integrate(ds.thetao, axis='Z')
hc_to = grid_to.integrate(theta_to, axis='Z')

(hc_fr == hc_to).all()

<xarray.DataArray ()>
array(True)